 Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [57]:
# Stochastic Gradient Descent Model

batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
        shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += 0.5 * beta * tf.nn.l2_loss(weights)
                                      
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)    
    

In [61]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {}".format(accuracy(
                valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 1942.63916016
Minibatch accuracy: 10.9375
Validation accuracy: 35.8
Minibatch loss at step 500: 127.059791565
Minibatch accuracy: 75.0
Validation accuracy: 81.99
Minibatch loss at step 1000: 10.8715686798
Minibatch accuracy: 88.28125
Validation accuracy: 84.72
Minibatch loss at step 1500: 1.41036760807
Minibatch accuracy: 87.5
Validation accuracy: 85.27
Minibatch loss at step 2000: 0.635771095753
Minibatch accuracy: 86.71875
Validation accuracy: 84.79
Minibatch loss at step 2500: 0.58384847641
Minibatch accuracy: 89.0625
Validation accuracy: 84.63
Minibatch loss at step 3000: 0.635760068893
Minibatch accuracy: 85.15625
Validation accuracy: 85.23
Test accuracy: 91.3


In [59]:
# Neural Network with 1 Hidden Layer
hidden_units = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
        shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Hidden Variables
    hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_units]))
    hidden_biases = tf.Variable(tf.zeros([hidden_units]))
    
    # relus
    tf_train_relus = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    tf_valid_relus = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    tf_test_relus = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    
    # Output Variables                                      
    weights = tf.Variable(
        tf.truncated_normal([hidden_units, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
                                       
    # Training computation
    logits = tf.matmul(tf_train_relus, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += 0.5 * beta * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights))
                                      
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_relus, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_relus, weights) + biases)    
    

In [60]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {}".format(accuracy(
                valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 1948.49157715
Minibatch accuracy: 14.84375
Validation accuracy: 42.31
Minibatch loss at step 500: 127.65309906
Minibatch accuracy: 74.21875
Validation accuracy: 82.46
Minibatch loss at step 1000: 10.9350214005
Minibatch accuracy: 86.71875
Validation accuracy: 85.22
Minibatch loss at step 1500: 1.40761351585
Minibatch accuracy: 88.28125
Validation accuracy: 85.0
Minibatch loss at step 2000: 0.617555141449
Minibatch accuracy: 86.71875
Validation accuracy: 84.7
Minibatch loss at step 2500: 0.586774587631
Minibatch accuracy: 89.0625
Validation accuracy: 84.63
Minibatch loss at step 3000: 0.636291265488
Minibatch accuracy: 85.15625
Validation accuracy: 85.26
Test accuracy: 91.4


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

**Minibatch accuracy jumps up to 100% becuase it's training itself perfectly on those few batch samples.**

---

In [62]:
# Stochastic Gradient Descent Model

batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
        shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += 0.5 * beta * tf.nn.l2_loss(weights)
                                      
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)    
    

In [63]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized
        offset = (step % 5 * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {}".format(accuracy(
                valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 34.4118118286
Minibatch accuracy: 7.03125
Validation accuracy: 9.71
Minibatch loss at step 500: 1.34485268593
Minibatch accuracy: 100.0
Validation accuracy: 74.32
Minibatch loss at step 1000: 0.306280642748
Minibatch accuracy: 100.0
Validation accuracy: 77.11
Minibatch loss at step 1500: 0.225412845612
Minibatch accuracy: 100.0
Validation accuracy: 77.32
Minibatch loss at step 2000: 0.218876108527
Minibatch accuracy: 100.0
Validation accuracy: 77.43
Minibatch loss at step 2500: 0.218251407146
Minibatch accuracy: 100.0
Validation accuracy: 77.41
Minibatch loss at step 3000: 0.21813762188
Minibatch accuracy: 100.0
Validation accuracy: 77.43
Test accuracy: 84.59


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [67]:
# Neural Network with 1 Hidden Layer
hidden_units = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
        shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder(tf.float32)
    
    # Hidden Variables
    hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_units]))
    hidden_biases = tf.Variable(tf.zeros([hidden_units]))
    
    # relus
    tf_train_relus = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases), keep_prob)
    tf_valid_relus = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    tf_test_relus = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    
    # Output Variables                                      
    weights = tf.Variable(
        tf.truncated_normal([hidden_units, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
                                       
    # Training computation
    logits = tf.matmul(tf_train_relus, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += 0.5 * beta * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights))
                                      
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_relus, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_relus, weights) + biases)    
    

In [68]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {}".format(accuracy(
                valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 2084.61254883
Minibatch accuracy: 8.59375
Validation accuracy: 35.93
Minibatch loss at step 500: 128.854873657
Minibatch accuracy: 71.875
Validation accuracy: 82.11
Minibatch loss at step 1000: 11.0565500259
Minibatch accuracy: 85.9375
Validation accuracy: 84.35
Minibatch loss at step 1500: 1.47641217709
Minibatch accuracy: 86.71875
Validation accuracy: 84.73
Minibatch loss at step 2000: 0.687007546425
Minibatch accuracy: 85.9375
Validation accuracy: 84.53
Minibatch loss at step 2500: 0.657440781593
Minibatch accuracy: 89.0625
Validation accuracy: 84.4
Minibatch loss at step 3000: 0.687291383743
Minibatch accuracy: 85.9375
Validation accuracy: 84.44
Test accuracy: 90.81


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [129]:
import math
image_size = 28
image_pixels = image_size * image_size
num_labels = 10
hidden1_units = 1024
hidden2_units = 512
    
# Neural Network with 1 Hidden Layer
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
        shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder(tf.float32)
    
    # Hidden 1
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal([image_pixels, hidden1_units],
            stddev=1.0 / math.sqrt(float(image_pixels))),
                            name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                            name='biases')
        hidden1_dict = {"weights": weights, "biases": biases}

    # Hidden 2
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
            stddev=1.0 / math.sqrt(float(hidden1_units))),
                            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                            name='biases')
        hidden2_dict = {"weights": weights, "biases": biases}
    
    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2_units, num_labels],
            stddev=1.0 / math.sqrt(float(hidden2_units))),
                            name='weights')
        biases = tf.Variable(tf.zeros([num_labels]),
                             name='biases')
        logits_dict = {"weights": weights, "biases": biases}
    
    def forward_prop(X, keep_prob):
        hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(X, hidden1_dict['weights']) +
                                    hidden1_dict['biases']), keep_prob)
        hidden2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden1, hidden2_dict['weights']) +
                                    hidden2_dict['biases']), keep_prob)
        return tf.matmul(hidden2, logits_dict['weights']) + logits_dict['biases']
    
    loss = tf.nn.softmax_cross_entropy_with_logits(forward_prop(tf_train_dataset, 0.5), tf_train_labels)
    loss += 0.5 * beta * (tf.nn.l2_loss(hidden1_dict['weights']) +
                          tf.nn.l2_loss(hidden2_dict['weights']) +
                          tf.nn.l2_loss(logits_dict['weights']))
    loss = tf.reduce_mean(loss)
    
    global_step = tf.Variable(0)   # count number of steps taken
    learning_rate = tf.train.exponential_decay(0.3, global_step, 3500, 0.86, staircase=True)
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(forward_prop(tf_train_dataset, 0.5))
    valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, 1))
    test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, 1))    
    

In [130]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
        _, l, predictions = session.run(
            [optimizer, loss[0], train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {}".format(accuracy(
                valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 5.35541439056
Minibatch accuracy: 17.96875
Validation accuracy: 40.57
Minibatch loss at step 500: 1.55858182907
Minibatch accuracy: 75.0
Validation accuracy: 84.17
Minibatch loss at step 1000: 0.810012936592
Minibatch accuracy: 86.71875
Validation accuracy: 84.45
Minibatch loss at step 1500: 0.618169903755
Minibatch accuracy: 88.28125
Validation accuracy: 84.97
Minibatch loss at step 2000: 0.605274677277
Minibatch accuracy: 87.5
Validation accuracy: 84.92
Minibatch loss at step 2500: 0.611281275749
Minibatch accuracy: 85.15625
Validation accuracy: 84.6
Minibatch loss at step 3000: 0.700701713562
Minibatch accuracy: 84.375
Validation accuracy: 85.31
Test accuracy: 91.68
